### Deploy Web App on Azure Container Services (AKS)
In this notebook we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the ACS cluster. Then we will check everything is working by sending an image to it and getting it scored.

The process is split into the following steps:
* [Define our resource names](#section1)
* [Login to Azure](#section2)
* [Create the ACS](#section3)
* [Create a tunnel to the head node](#section4)
* [Create a JSON schema of our APP and push it to the cluster](#section5)
* [Test our app](TestWebApp.ipynb)
* [Tear it all down](#section7)

This guide assumes is designed to be run on linux and requires that the Azure CLI is installed.

<a id='section1'></a>

## Setup
Below are the various name definitions for the resources needed to setup ACS as well as the name of the Docker image we will be using.

**Some outputs (and inputs) below have been hidden/masked for confidentiality**

In [24]:
resource_group = "msaksrg" # Feel free to modify these
acs_name = "msaks"
location = "eastus"

image_name = 'masalvar/cntkresnet-gpu' 
selected_subscription = "'Team Danielle Internal'" # If you have multiple subscriptions select 
                                                   # the subscription you want to use here

<a id='section2'></a>

## Azure account login
The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [2]:
!az login -o table

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FYXCVTLFG to authenticate.
CloudName    Name                           State    TenantId                              IsDefault
-----------  -----------------------------  -------  ------------------------------------  -----------
AzureCloud   Boston DS Dev                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Azure Internal - London        Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Team Danielle Internal         Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47  True
AzureCloud   Visual Studio Enterprise       Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Boston Engineering             Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   ADLTrainingMS                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   PhillyExt                      Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Ads Eng Big Data Sub

In [3]:
!az account set --subscription $selected_subscription

In [25]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "masalvar@microsoft.com",
    "type": "user"
  }
}


In [5]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


<a id='section3'></a>

## Create resources and dependencies

### Create resource group
Azure encourages the use of groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the group.

In [6]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/msaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "msaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


In [26]:
resource_group

'msaksrg'

In [27]:
!az aks create --resource-group $resource_group --name myAKSCluster --node-count 1 --generate-ssh-keys -s Standard_NC6

{- Finished ..
  "additionalProperties": {},
  "agentPoolProfiles": [
    {
      "additionalProperties": {},
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "myAKSClust-msaksrg-edf507",
  "fqdn": "myaksclust-msaksrg-edf507-823b8e55.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msaksrg/providers/Microsoft.ContainerService/managedClusters/myAKSCluster",
  "kubernetesVersion": "1.7.9",
  "linuxProfile": {
    "additionalProperties": {},
    "adminUsername": "azureuser",
    "ssh": {
      "additionalProperties": {},
      "publicKeys": [
        {
          "additionalProperties": {},
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDVfKBWPBKS84wluD3DJ0t3hepO2F13pz1VI5d4c7Tn4d80rSKJkF2L2H

Install cli from comamnd prompt

In [9]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


In [23]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "masalvar/cntkresnet-gpu",
                      "ports":[
                          {
                              "containerPort":88,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "name": "bin",
                              "mountPath":"/usr/local/nvidia/bin" 
                          },
                          {
                              "name": "lib",
                              "mountPath":"/usr/local/nvidia/lib64" 
                          },
                          {
                              "name": "libcuda",
                              "mountPath":"/usr/lib/x86_64-linux-gnu/libcuda.so.1" 
                          },
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }
                      
                      
                  }
              ],
              "volumes":[
                  {
                      "name": "bin",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384/bin"
                      },
                  },
                  {
                      "name": "lib",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384"
                      },
                  },
                  {
                      "name": "libcuda",
                      "hostPath":{
                          "path":"/usr/lib/x86_64-linux-gnu/libcuda.so.1"
                      },
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "ports":[
          {
              "port":88,
              "selector":{
                  "app":"azure-dl"
              }
          }
      ]
  }
}

In [42]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "masalvar/cntkresnet-gpu",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "name": "bin",
                              "mountPath":"/usr/local/nvidia/bin" 
                          },
                          {
                              "name": "lib",
                              "mountPath":"/usr/local/nvidia/lib64" 
                          },
                          {
                              "name": "libcuda",
                              "mountPath":"/usr/lib/x86_64-linux-gnu/libcuda.so.1" 
                          },
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "bin",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384/bin"
                      },
                  },
                  {
                      "name": "lib",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384"
                      },
                  },
                  {
                      "name": "libcuda",
                      "hostPath":{
                          "path":"/usr/lib/x86_64-linux-gnu/libcuda.so.1"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [43]:
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4,sort_keys=True)
        outfile.write('\n\n')

In [44]:
write_json_to_file(app_template, 'az-dl.json')

In [45]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

In [46]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                            }
                        ],
                        "image": "masalvar/cntkresnet-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
          

In [33]:
!az aks get-credentials --resource-group=$resource_group --name=myAKSCluster

Merged "myAKSCluster" as current context in /home/mat/.kube/config


In [47]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-12483742-0   Ready     agent     29m       v1.7.9


In [48]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   heapster-2574232661-103s7               2/2       Running   0          27m
kube-system   kube-dns-v20-2253765213-d8mg7           3/3       Running   0          28m
kube-system   kube-dns-v20-2253765213-qw371           3/3       Running   0          28m
kube-system   kube-proxy-lhd13                        1/1       Running   0          28m
kube-system   kube-svc-redirect-7bqsm                 1/1       Running   0          28m
kube-system   kubernetes-dashboard-2898242510-pvk5h   1/1       Running   0          28m
kube-system   tunnelfront-3600233641-xg9z3            1/1       Running   0          28m


In [49]:
!kubectl create -f az-dl.json

deployment "azure-dl" created
service "azure-dl" created


In [53]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-2914933029-vdtbv               1/1       Running   0          12h
kube-system   heapster-2574232661-103s7               2/2       Running   0          13h
kube-system   kube-dns-v20-2253765213-d8mg7           3/3       Running   0          13h
kube-system   kube-dns-v20-2253765213-qw371           3/3       Running   0          13h
kube-system   kube-proxy-lhd13                        1/1       Running   0          13h
kube-system   kube-svc-redirect-7bqsm                 1/1       Running   0          13h
kube-system   kubernetes-dashboard-2898242510-pvk5h   1/1       Running   0          13h
kube-system   tunnelfront-3600233641-xg9z3            1/1       Running   0          13h


In [52]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                        KIND      SUBOBJECT   TYPE      REASON                             SOURCE                              MESSAGE
10m         13h          6         aks-nodepool1-12483742-0.151d1c455b4c4741   Node                  Normal    NodeHasSufficientDisk              kubelet, aks-nodepool1-12483742-0   Node aks-nodepool1-12483742-0 status is now: NodeHasSufficientDisk
10m         13h          6         aks-nodepool1-12483742-0.151d1c455b4c84fb   Node                  Normal    NodeHasSufficientMemory            kubelet, aks-nodepool1-12483742-0   Node aks-nodepool1-12483742-0 status is now: NodeHasSufficientMemory
10m         13h          6         aks-nodepool1-12483742-0.151d1c455b4ca4a0   Node                  Normal    NodeHasNoDiskPressure              kubelet, aks-nodepool1-12483742-0   Node aks-nodepool1-12483742-0 status is now: NodeHasNoDiskPressure
1s          13h          802       aks-nodepool1-12483742-0.1

In [54]:
!kubectl logs azure-dl-2914933029-vdtbv

2018-03-18 20:34:48,373 CRIT Supervisor running as root (no user in config file)
2018-03-18 20:34:48,375 INFO supervisord started with pid 7
2018-03-18 20:34:49,377 INFO spawned: 'program_exit' with pid 17
2018-03-18 20:34:49,379 INFO spawned: 'nginx' with pid 18
2018-03-18 20:34:49,380 INFO spawned: 'gunicorn' with pid 19
2018-03-18 20:34:50,409 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-03-18 20:34:54,417 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
Selected GPU[0] Tesla K80 as the process wide default device.
Initialising
Model loading time: 12839.98 ms
{"timestamp": "2018-03-18T20:35:02.836512Z", "message": "Model loading time: 12839.98 ms", "host": "azure-dl-2914933029-vdtbv", "path": "/code/driver.py", "tags": [], "level": "INFO", "logger": "cntk_svc_logger", "stack_info": null}
{"timestamp": "2018-03-18T20:35:02.840847Z", "message": " * Running on http://127.0.

{"timestamp": "2018-03-18T20:39:18.927960Z", "message": "Time distribution: preprocess=3.45 ms, eval=64.46 ms, resultPrep = 0.1 ms", "host": "azure-dl-2914933029-vdtbv", "path": "/code/driver.py", "tags": [], "level": "INFO", "logger": "cntk_svc_logger", "stack_info": null}
{"timestamp": "2018-03-18T20:39:18.928439Z", "message": "127.0.0.1 - - [18/Mar/2018 20:39:18] \"POST /score HTTP/1.0\" 200 -", "host": "azure-dl-2914933029-vdtbv", "path": "/opt/conda/envs/py3.6/lib/python3.6/site-packages/werkzeug/_internal.py", "tags": [], "level": "INFO", "logger": "werkzeug", "stack_info": null}
Predictions: [[('n02690373 airliner', 2057.77645111084), ('n04592741 wing', 1766.9858932495117), ('n04266014 space shuttle', 1649.7426986694336)]]
{"timestamp": "2018-03-18T20:40:03.205950Z", "message": "Predictions: [[('n02690373 airliner', 2057.77645111084), ('n04592741 wing', 1766.9858932495117), ('n04266014 space shuttle', 1649.7426986694336)]]", "host": "azure-dl-2914933029-vdtbv", "path": "/code/dr

{"timestamp": "2018-03-18T20:40:05.580447Z", "message": "Time distribution: preprocess=3.27 ms, eval=63.89 ms, resultPrep = 0.09 ms", "host": "azure-dl-2914933029-vdtbv", "path": "/code/driver.py", "tags": [], "level": "INFO", "logger": "cntk_svc_logger", "stack_info": null}
{"timestamp": "2018-03-18T20:40:05.580947Z", "message": "127.0.0.1 - - [18/Mar/2018 20:40:05] \"POST /score HTTP/1.0\" 200 -", "host": "azure-dl-2914933029-vdtbv", "path": "/opt/conda/envs/py3.6/lib/python3.6/site-packages/werkzeug/_internal.py", "tags": [], "level": "INFO", "logger": "werkzeug", "stack_info": null}
Predictions: [[('n01530575 brambling, Fringilla montifringilla', 1367.5324440002441), ('n01807496 partridge', 963.9557838439941), ('n01601694 water ouzel, dipper', 886.3053321838379)]]
{"timestamp": "2018-03-18T20:40:05.842165Z", "message": "Predictions: [[('n01530575 brambling, Fringilla montifringilla', 1367.5324440002441), ('n01807496 partridge', 963.9557838439941), ('n01601694 water ouzel, dipper', 

In [55]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP   EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.125.6   52.234.213.11   80:30179/TCP   12h


In [157]:
# from time import sleep
# for i in range(20):
#     json_data = !curl http://localhost:1212/marathon/v2/apps
#     if json.loads(json_data[-1])['apps'][0]['tasksRunning']==1:
#         print('Web app ready')
#         break
#     else:
#         print('Preparing Web app')
#     sleep(10)
# else:
#     print('Timeout! Something went wrong!')

Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Preparing Web app
Web app ready


<a id='section7'></a>

## Tear it all down 
Once you are done with your cluster you can use the following two commands to destroy it all.

In [ ]:
!kubectl delete -f az-dl.json

In [ ]:
!aks delete -n myAKSCluster -g $resource_group

In [162]:
!az group delete --name $resource_group -y